In [5]:
import torch
from torch import nn
import torch.nn.functional as F

import numpy as np
import os
import sys
sys.path.append('../')

import pyro
import pyro.distributions as dist

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
data_dir = 

In [7]:
from pyro.nn import PyroSample, PyroModule


class BayesianRegression(PyroModule):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = PyroModule[nn.Linear](in_features, out_features)
        self.linear.weight = PyroSample(dist.Normal(0., 1.).expand([out_features, in_features]).to_event(2))
        self.linear.bias = PyroSample(dist.Normal(0., 10.).expand([out_features]).to_event(1))

    def forward(self, x, y=None):
        sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
        mean = self.linear(x).squeeze(-1)
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mean, sigma), obs=y)
        return mean